# Build Meter Table
Build the meters table from extracted data as well as location and wells tables.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load extracted_meters, locations, and wells
meters = pd.read_csv('./api/data/extracted_meters.csv')
locations = pd.read_csv('./api/data/locations.csv')
wells = pd.read_csv('./api/data/wells.csv')

Steps:
* Associate meters with wells/locations using the RA number
* Add any undefined wells based on installed meters with lat/lon
* If meter appears to be in wearhouse, add location

In [3]:
# Join well_id and location_id from wells to meters based on RA number
meters_locations = pd.merge(meters, wells[['name','well_id', 'location_id']], how='left', left_on='sRANumber', right_on='name')

In [9]:
# Assess installed meters without well_id
meters_locations[meters_locations['meter_status_id'] == 1 & meters_locations['well_id'].isna()]

,sSerialNumber,sContact,sTag,sRANumber,sTRSS,owner_id,meter_type_id,meter_status_id,latitude,longitude,name,well_id,location_id
46,00-6-3949,TIM OR ELIZABETH ARGO,207E5,NaN,9.24.14.431,NaN,2.0,1,33.529278,104.478167,NaN,NaN,NaN
1648,17-9-06604,BERRENDO WATER USERS ASSOC.,NaN,NaN,11.23.4.122,NaN,14.0,1,33.393306,104.623806,NaN,NaN,NaN
1748,19-4-00254,BILL HORNER,NaN,NaN,WAREHOUSE,16.0,1.0,1,32.864750,104.362583,NaN,NaN,NaN
1749,19-4-00255,CHISUM TRAIL DAIRY,NaN,NaN,12.25.35.1333,NaN,1.0,1,NaN,NaN,NaN,NaN,NaN


Six installed meters don't have matching wells/locations. Before adding any as new wells, I need to check if the sTRSS is in the locations table.

In [7]:
missing_wells = meters_locations[meters_locations['meter_status_id'] == 1 & meters_locations['well_id'].isna()].loc[:,['sSerialNumber','sTRSS']]

#Merge missing_wells with locations based on TRSS
pd.merge(missing_wells, locations[['sTRSS','location_id']], how='left', left_on='sTRSS', right_on='sTRSS')


,sSerialNumber,sTRSS,location_id
0,00-6-3949,9.24.14.431,NaN
1,17-9-06604,11.23.4.122,NaN
2,19-4-00254,WAREHOUSE,630.0
3,19-4-00255,12.25.35.1333,NaN
4,33176703,5206 W. 2ND ST.,634.0
5,853121,0,686.0


The serial numbers with sTRSS appear to be new locations, but given it is only three wells/locations, I think it will be easier for PVACD to work this out and will leave them without locations.

I suspect the last two rows are actually uninstalled or sold meters, so I will simply change their status to 'unknown'.

In [8]:
meters_locations.loc[2076,'meter_status_id'] = 6
meters_locations.loc[2939,'meter_status_id'] = 6

Meters are finally ready for saving to CSV!

In [10]:
meters_table = meters_locations.loc[:,['sSerialNumber','sContact','sTag','meter_type_id','meter_status_id','well_id','location_id']].sort_values(by=['sSerialNumber'])

# Add on meter_id
meters_table['meter_id'] = np.arange(1, len(meters_table)+1)


In [11]:
# Rename sSerialNumber to serial_number
meters_table.rename(columns={'sSerialNumber':'serial_number'}, inplace=True)

# Rename sContact to old_contact_name
meters_table.rename(columns={'sContact':'old_contact_name'}, inplace=True)

# Rename sTag to tag
meters_table.rename(columns={'sTag':'tag'}, inplace=True)

In [12]:
# Reorder columns to 'meter_id','serial_number','old_contact_name','tag','meter_type_id','meter_status_id','location_id','well_id'
meters_table = meters_table.loc[:,['meter_id','serial_number','old_contact_name','tag','meter_type_id','meter_status_id','location_id','well_id']]

In [13]:
# Save meters_table to csv
meters_table.to_csv('./api/data/meters.csv', index=False)